In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import imageio
from mpi4py import MPI

In [ ]:
# Generate Random image
r = 150
c = 200
np.random.seed(55)
picture = np.random.choice(range(0, 5),size=(r,c), p=[0.2,0.15,0.15,0.1,0.4])
fig = plt.figure(figsize = (12,6))
plt.imshow(picture, cmap='Purples' )
plt.savefig('pic.png')

In [ ]:
!mpiexec -n 4 python Task8.py

In [ ]:
# Save npy arrays
res = []
for i in range(4):
    with open(str(i)+'.npy', 'rb') as f:
        res.append(np.load(f))
res = np.concatenate(res,axis = 1)

In [ ]:
res.shape

In [ ]:
# Save col-sifted images to later make gif
!mkdir picture
for i,step in enumerate(res):
    fig = plt.figure(figsize = (14,11))
    plt.imshow(step, cmap='Purples' )
    if i < 10:
        plt.savefig('picture/'+'00'+str(i)+'.png')
    elif i < 100:
        plt.savefig('picture/'+'0'+str(i)+'.png')
    else:
        plt.savefig('picture/'+str(i)+'.png')
    plt.close(fig)

In [ ]:
# make gif, display it
from os import listdir
images = []
for filename in sorted(listdir('picture')):
    images.append(imageio.imread('picture/'+filename))
imageio.mimsave('picture/picture.gif', images)

In [ ]:
from IPython.display import Image
Image(filename="picture/picture.gif")

In [ ]:
!mpiexec -n 2 python -m memory_profiler Task8.py

# Cython Optimization

#### Step 1 Create python file: Task8.py

In [ ]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
r = 150
c = 200
np.random.seed(55)
picture = np.random.choice(range(0, 5),size=(r,c), p=[0.2,0.15,0.15,0.1,0.4])

if size > r:
    size = r
if rank<size:
    index_start = int(rank * r / size)
    index_end = int((rank+1) * r / size)
    #print(index_start,index_end)
    if rank == size - 1:
        index_end = r
    steps = []
    picture = picture[index_start:index_end]
    steps = [picture.copy()]
    for _ in range(c):
        for row in range(picture.shape[0]):
            tmp = picture[row,0]
            for column in range(c-1):
                picture[row,column] = picture[row,column+1]
            picture[row,c-1] = tmp
        steps.append(picture.copy())
    steps = np.array(steps)
    with open(str(rank)+'.npy', 'wb') as f:
        np.save(f, steps)

#### Step 2 : Create cython pyx file

In [ ]:
# Before loading,  let's see how much we've optimized
# Load cython extension
%load_ext cython

In [ ]:
%%cython -a
from mpi4py import MPI
import numpy as np

cpdef run():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    cdef int r = 150
    cdef int c = 200
    np.random.seed(55)
    cdef int [:,:] picture = np.random.choice(range(0, 5),size=(r,c), p=[0.2,0.15,0.15,0.1,0.4])
    #int [r, c] pic_c = picture
    steps = []
    cdef int tmp 
    if size > r:
        size = r
    if rank<size:
        index_start = <int>(rank * r / size)
        index_end = <int>((rank+1) * r / size)
        #print(index_start,index_end)
        if rank == size - 1:
            index_end = r
        
        picture = picture[index_start:index_end]
        steps = [picture.copy()]
        for _ in range(c):
            for row in range(picture.shape[0]):
                tmp = picture[row,0]
                for column in range(c-1):
                    picture[row,column] = picture[row,column+1]
                picture[row,c-1] = tmp
            steps.append(picture.copy())
        steps = np.array(steps)
        with open(str(rank)+'.npy', 'wb') as f:
            np.save(f, steps)

In [ ]:
%%writefile task8_cy.pyx
from mpi4py import MPI
import numpy as np

cpdef run():
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    cdef int r = 150
    cdef int c = 200
    np.random.seed(55)
    cdef int [:,:] picture = np.random.choice(range(0, 5),size=(r,c), p=[0.2,0.15,0.15,0.1,0.4])
    #int [r, c] pic_c = picture
    steps = []
    cdef int tmp 
    if size > r:
        size = r
    if rank<size:
        index_start = <int>(rank * r / size)
        index_end = <int>((rank+1) * r / size)
        #print(index_start,index_end)
        if rank == size - 1:
            index_end = r
        
        picture = picture[index_start:index_end]
        steps = [picture.copy()]
        for _ in range(c):
            for row in range(picture.shape[0]):
                tmp = picture[row,0]
                for column in range(c-1):
                    picture[row,column] = picture[row,column+1]
                picture[row,c-1] = tmp
            steps.append(picture.copy())
        steps = np.array(steps)
        with open(str(rank)+'.npy', 'wb') as f:
            np.save(f, steps)

#### Step 3 Create Setup file 

In [ ]:
%%writefile setup_task8.py
from setuptools import setup
from Cython.Build import cythonize

setup(
    name='Columnwise Shift',
    ext_modules=cythonize("task8_cy.pyx"),
    zip_safe=False,
)

#### Step 4 Build the cython file

In [ ]:
!python setup_task8.py build_ext --inplace

In [ ]:
%%writefile task8_run.py
import task8_cy
task8_cy.run()
#This is the file that will be used for mpi calls

In [ ]:
%timeit !python task8_run.py
%timeit !python Task8.py


In [ ]:
res_py = []
res_cy = []
for i in range(1, 5):
    res = %timeit -o !mpiexec -n {i} python Task8.py
    res_py.append(res.average)
    res = %timeit -o !mpiexec -n {i} python task8_run.py
    res_cy.append(res.average)

In [ ]:
%pylab inline
#times = [1 / x * times[0] for x in times]
figure(figsize = (16,8))
plot(range(1,len(res_py)+1,1), res_py,label = 'Python')
plot(range(1,len(res_py)+1,1), res_cy,label = 'Cython')
#plot(range(1,len(times)+1), range(1,len(times)+1),label='desired')
plt.title("Acceleration vs number of processes")
plt.xlabel('Number of processes')
plt.ylabel('Acceleration')
plt.grid()
plt.legend()
plt.show()

In [ ]:
import tracemalloc
from tqdm import tqdm

In [ ]:
num_processes = [1,2,3,4]
curs_py = []
peaks_py = []
curs_cy = []
peaks_cy = []
#tracemalloc.start()
for n in num_processes:
    tracemalloc.start()
    for i in range(2**20):
        pass
    
    !mpiexec -n {n} python Task8.py
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    peaks_py.append(peak)
    curs_py.append(current)
    tracemalloc.start()
    for i in range(2**20):
        pass
    !mpiexec -n {n} python task8_run.py
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    peaks_cy.append(peak)
    curs_cy.append(current)

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(num_processes, peaks_cy, linewidth=2, label='python')
plt.plot(num_processes, peaks_py, linewidth=2, label='cython')
plt.legend(('cython', 'python'))
plt.title("Memory consumption", fontsize=15)
plt.xlabel("№ of processes")
plt.ylabel("Memory Consumption")
plt.grid(True)

# Conclusion
1. Using MPI4PY resulted in an increase in speed proportional to the number of processes spawned
2. However, the total memory dropped from 1 process to two, then remained constant thereafter.